## Setup

In the Terminal:

1) Create a virtual environment and open it
To create a virtual env:
```
$ python -m venv /path/to/new/virtual/environment
```
To open it
windows:

mac:
source venv/bin/activate

2) pip install the following packages
```
$ pip install matplotlib, numpy==1.26.0, netCDF4, cartopy, xarray
```

## Exercise 1a): Matplotlib advanced features: inset plots

This execise demonstrates adding subplots to an axes

Try changing which section of the main plot the inset captures, and moving the position of the inset

In [ ]:
import matplotlib.figure as figure
import matplotlib.backends.backend_agg as agg
import numpy as np
import matplotlib.pyplot as plt

# Create a figure
fig = figure.Figure(figsize=(6, 4))
canvas = agg.FigureCanvasAgg(fig)

# Add main axes
main_ax = fig.add_axes([0.1, 0.1, 0.7, 0.7])

# Add inset axes
inset_ax = fig.add_axes([0.55, 0.55, 0.3, 0.3])

# Data for main plot
x_main = np.linspace(0, 10, 100)
y_main = np.sin(x_main)

# Data for inset plot (highlight a specific region of the main data)
x_inset = np.linspace(2, 4, 50)
y_inset = np.sin(x_inset)

# Plot data on main axes
main_ax.plot(x_main, y_main, label='Main Data')
main_ax.set_title('Main Plot')
main_ax.set_xlabel('X-axis')
main_ax.set_ylabel('Y-axis')
main_ax.legend()

# Plot data on inset axes
inset_ax.plot(x_inset, y_inset, color='red', label='Inset Data')
inset_ax.set_title('Inset Plot')
inset_ax.legend()

# Highlight the inset area on the main plot
main_ax.indicate_inset_zoom(inset_ax)

# Render the plot
canvas.draw()

# Convert to a displayable format
s, (width, height) = canvas.print_to_buffer()
buffer = np.frombuffer(s, np.uint8).reshape((height, width, 4))

plt.imshow(buffer)
plt.axis('off')
plt.show()


## Exercise 2a) Custom date time axes formats

Its common to plot time series data using Matplotlib. This section explores the default date formatter and shows alternatives which may be more readable.

In [ ]:
import matplotlib.cbook as cbook
import matplotlib.dates as mdates

# Load a numpy record array from yahoo csv data with fields date, open, high,
# low, close, volume, adj_close from the mpl-data/sample_data directory. The
# record array stores the date as an np.datetime64 with a day unit ('D') in
# the date column.
data = cbook.get_sample_data('goog.npz')['price_data']

fig, axs = plt.subplots(3, 1, figsize=(6.4, 7), layout='constrained')
# common to all three:
for ax in axs:
    ax.plot('date', 'adj_close', data=data)
    # Major ticks every half year, minor ticks every month,
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.grid(True)
    ax.set_ylabel(r'Price [\$]')

# different formats:
ax = axs[0]
ax.set_title('DefaultFormatter', loc='left', y=0.85, x=0.02, fontsize='medium')

ax = axs[1]
ax.set_title('ConciseFormatter', loc='left', y=0.85, x=0.02, fontsize='medium')
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

ax = axs[2]
ax.set_title('Manual DateFormatter', loc='left', y=0.85, x=0.02,
             fontsize='medium')
# Text in the x-axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
# Rotates and right-aligns the x labels so they don't crowd each other.
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

plt.show()

## Exercise 2c) Annotating plots

It can be useful to highlight specific data areas or add commentry. Matplotlib supports a subset of latex which can be used to dsiplay mathemeatics or text.

Try moving the annotations and modifying their content.

In [ ]:
# Create a figure
fig, main_ax = plt.subplots(figsize=(6, 4))

# Data for main plot
x_main = np.linspace(0, 10, 100)
y_main = np.sin(x_main)

# Plot data on main axes
main_ax.plot(x_main, y_main, label='Main Data')
main_ax.set_title('Main Plot')
main_ax.set_xlabel('X-axis')
main_ax.set_ylabel('Y-axis')
main_ax.legend()

# Add the first annotation
main_ax.annotate('Peak', xy=(np.pi/2, 1), xytext=(4, 0.5),
                 arrowprops=dict(facecolor='black', shrink=0.05))

# Add the second annotation
main_ax.annotate('y = Sin(x)', xy=(3*np.pi/2, -1), xytext=(4, 0),
                 arrowprops=dict(facecolor='red', shrink=0.05))

# Display the plot
plt.show()


## Exercise 3a) Plotting NetCDF data

NetCDF data needs to be processed before it can be plotted effectively in matplotlib.

The first codebox simply displays the NetCDF data we are going to plot. Find the following:
- source 
- temporal range
- extents of longitude and latitude
- variables

In [ ]:
import netCDF4 as nc

# Open the NetCDF file
file_path = '../data/MERRA2_400.tavg1_2d_lnd_Nx.20240401.nc4'
dataset = nc.Dataset(file_path)

# Print metadata
print(dataset)

# Close the dataset
dataset.close()

## Exercise 3b) Plot the netCDF data

Try changing the colour scheme or changing the plot to focus on a smaller region of the earth

For this exercise we will be using `dataset` as loaded above.

In [ ]:
# Load the netCDF file
file_path = '../data/MERRA2_400.tavg1_2d_lnd_Nx.20240401.nc4'
dataset = nc.Dataset(file_path)

# Extract the variables
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]
temps = dataset.variables['TSURF'][0, :, :]  # Assuming the time dimension is the first dimension

# Close the dataset
dataset.close()

# Create a figure
fig, ax = plt.subplots(figsize=(12, 6))

# Plot the temperature data
c = ax.pcolormesh(lons, lats, temps, shading='auto')
fig.colorbar(c, ax=ax, label='Temperature')

# Set labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Surface Temperature Distribution')

plt.show()

## Excercise 4a) Cartopy coastlines

In [ ]:
import cartopy.crs as ccrs

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.show()

## Exercise 4b) Cartopy features

In [ ]:
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([80, 170, -45, 30])

# Put a background image on for nice sea rendering.
ax.stock_img()

# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

SOURCE = 'Natural Earth'
LICENSE = 'public domain'

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(states_provinces, edgecolor='gray')

#Add a text annotation for the license information to the
#the bottom right corner.
text = AnchoredText('\u00A9 {}; license: {}'
                        ''.format(SOURCE, LICENSE),
                        loc=4, prop={'size': 8}, frameon=True)
ax.add_artist(text)

plt.show()



## Exercise 4c) Cartopy projections

Try:
- rotating the plot
- changing the projection (https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html)
- changing the colourmap

In [ ]:
# Open the NetCDF file
file_path = '../data/MERRA2_400.tavg1_2d_lnd_Nx.20240401.nc4'
dataset = nc.Dataset(file_path)

# Extract the variables
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]
temps = dataset.variables['TSURF'][0, :, :]  # Assuming the time dimension is the first dimension

# Close the dataset
dataset.close()

# Create a figure
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': ccrs.Orthographic(central_longitude=0, central_latitude=50)})

# Plot the temperature data
c = ax.pcolormesh(lons, lats, temps, shading='auto', transform=ccrs.PlateCarree(), cmap='coolwarm')
fig.colorbar(c, ax=ax, label='Temperature')

# Add features for better visualization
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)

# Set title
ax.set_title('Surface Temperature Distribution')

plt.show()


## Exercise 5) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# Open the NetCDF file
dataset = xr.load_dataset("../data/ERA5_vector_data.nc", engine="netcdf4")

# Extract the variables
u = dataset.variables['u10'][0, :, :].values  # U-component of wind
v = dataset.variables['v10'][0, :, :].values  # V-component of wind
lats = dataset.variables['latitude'][:].values
lons = dataset.variables['longitude'][:].values

# Close the dataset
dataset.close()

# Calculate the magnitude of the wind vectors
magnitude = (u ** 2 + v ** 2) ** 0.5

# Normalize the magnitude for coloring
norm = Normalize(vmin=np.min(magnitude), vmax=np.max(magnitude))
cmap = cm.viridis

# Plotting
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0, central_latitude=50))

# Create the streamlines with color mapping based on magnitude
strm = ax.streamplot(lons, lats, u, v, transform=ccrs.PlateCarree(), color=magnitude, linewidth=1, cmap=cmap, norm=norm, density=1.5)

# Add color to the land and sea
land = cfeature.NaturalEarthFeature(
    'physical', 'land', '110m',
    edgecolor='face',
    facecolor=cfeature.COLORS['land']
)
ax.add_feature(land)

sea = cfeature.NaturalEarthFeature(
    'physical', 'ocean', '110m',
    edgecolor='face',
    facecolor=cfeature.COLORS['water']
)
ax.add_feature(sea)

# Add coastlines
ax.coastlines(color='r', linewidth=1)

# Add a color bar with adjusted size
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(magnitude)
plt.colorbar(sm, ax=ax, orientation='horizontal', label='Wind Speed (m/s)', shrink=0.5, pad=0.1)

plt.show()


In [ ]:
from matplotlib.colors import Normalize
import matplotlib.animation as animation
from IPython import display

# Open the NetCDF file
dataset = xr.load_dataset("../data/ERA5_vector_data.nc", engine="netcdf4")

# Define the latitude and longitude bounds for the UK and Ireland
lat_bounds = [49, 61]
lon_bounds = [-12, 4]

# Extract latitude and longitude arrays
lats = dataset.variables['latitude'][:].values
lons = dataset.variables['longitude'][:].values

# Convert longitudes from [0, 360] to [-180, 180]
lons = np.where(lons > 180, lons - 360, lons)

# Find the indices that correspond to the desired lat/lon bounds
lat_indices = np.where((lats >= lat_bounds[0]) & (lats <= lat_bounds[1]))[0]
lon_indices = np.where((lons >= lon_bounds[0]) & (lons <= lon_bounds[1]))[0]

# Slice the data to include only these bounds
u = dataset.variables['u10'][:, lat_indices, :][:, :, lon_indices].values
v = dataset.variables['v10'][:, lat_indices, :][:, :, lon_indices].values
lats = lats[lat_indices]
lons = lons[lon_indices]

# Set up the plot
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-12, 4, 49, 61], crs=ccrs.PlateCarree())  # Focus on UK and Ireland

# Add color to the land and sea
land = cfeature.NaturalEarthFeature(
    'physical', 'land', '110m',
    edgecolor='face',
    facecolor=cfeature.COLORS['land']
)
ax.add_feature(land)

sea = cfeature.NaturalEarthFeature(
    'physical', 'ocean', '110m',
    edgecolor='face',
    facecolor=cfeature.COLORS['water']
)
ax.add_feature(sea)

# Add coastlines
ax.coastlines(color='r', linewidth=1)

# Normalize the magnitude for coloring
norm = Normalize(vmin=np.min(u**2 + v**2), vmax=np.max(u**2 + v**2))
cmap = cm.viridis

# Function to update the streamline plot
def update(frame):
    ax.clear()  # Clear the previous streamlines
    ax.set_extent([-12, 4, 49, 61], crs=ccrs.PlateCarree())  # Reset the extent
    ax.add_feature(land)
    ax.add_feature(sea)
    ax.coastlines(color='r', linewidth=1)

    magnitude = (u[frame, :, :] ** 2 + v[frame, :, :] ** 2) ** 0.5
    strm = ax.streamplot(lons, lats, u[frame, :, :], v[frame, :, :], 
                         transform=ccrs.PlateCarree(), color=magnitude, linewidth=2, cmap=cmap, norm=norm)
    return strm

# Create animation
ani = animation.FuncAnimation(fig, update, frames=len(dataset.variables['time']), blit=False, interval=1000)

# Close the dataset
dataset.close()

# Assign the animation to a variable to prevent it from being deleted
anim = ani

# converting to an html5 video 
video = anim.to_html5_video() 
  
# embedding for the video 
html = display.HTML(video) 
  
# draw the animation 
display.display(html) 
